In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from sklearn.datasets import make_classification
import numpy as np
import tensorflow as tf

In [2]:
def make_dummy_dataset(n_samples=1000):
    X, y = make_classification(n_classes=2,
                               n_samples=n_samples,
                               n_informative=3,
                               n_repeated=0,
                               n_redundant=0,
                               n_features=3,
                               random_state=42)
    data = np.concatenate([X, np.expand_dims(y, axis=-1)], axis=1)
    return data

In [3]:
with open('data.csv', 'w') as f:
    import csv

    writer = csv.writer(f)
    writer.writerows(make_dummy_dataset())

In [4]:
def parse(a_line):
    splits = tf.strings.split(a_line, ',')
    X = [float(splits[0]), float(splits[1]), float(splits[2])]
    y = float(splits[3])
    return (X, y)


dataset = tf.data.TextLineDataset('data.csv').map(parse).cache().shuffle(10000).batch(32).prefetch(2)

In [5]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=[3]),
    tf.keras.layers.Dense(units=8, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')
])
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])
model.fit(dataset, epochs=2)

Epoch 1/2
32/32 [==============================] - 1s 19ms/step - loss: 0.8498 - accuracy: 0.4090
Epoch 2/2
32/32 [==============================] - 0s 4ms/step - loss: 0.7784 - accuracy: 0.4870
